In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import preprocessing.generate_preprocessed_dataset as pp
from training.temp_attention_training import run_temp_model_training
import time

In [3]:
# X, y, groups, activities = pp.preprocessing(n_epochs=5)

In [4]:
# X.shape, y.shape, groups.shape, activities.shape

In [ ]:
start_time = time.time()
run_temp_model_training(16000)
end_time = time.time()
print(f"Training completed in {(end_time - start_time) / 3600:.2f} hours.")

The device used for training is: cuda
Epoch 1/500, Loss: 928.8346887403919
Validation Loss: 352.713732819059
Epoch 2/500, Loss: 291.7452853623257
Validation Loss: 384.2623309235075
Epoch 3/500, Loss: 254.11762459047378
Validation Loss: 360.79875251428405
Epoch 4/500, Loss: 228.68563301332534
Validation Loss: 364.5790500925548
Epoch 5/500, Loss: 215.29760504281649
Validation Loss: 464.28519587730295
Epoch 6/500, Loss: 204.23540841379474
Validation Loss: 464.8714435634328
Epoch 7/500, Loss: 195.73880817044167
Validation Loss: 525.2250644057544
Epoch 8/500, Loss: 190.69241464266213
Validation Loss: 514.7218295424732
Epoch 9/500, Loss: 185.62028060420866
Validation Loss: 517.3043477072645
Epoch 10/500, Loss: 181.19141100811703
Validation Loss: 511.213708222802
Epoch 11/500, Loss: 178.5207995342952
Validation Loss: 454.47804146382344
Epoch 12/500, Loss: 171.72897170692363
Validation Loss: 512.0516580610133
Epoch 13/500, Loss: 170.4824925904633
Validation Loss: 569.0164530739855
Epoch 14/500